In [1]:
!pip install numpy scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 6.6 MB/s eta 0:00:0000:0100:01


In [2]:
import pandas as pd
import numpy as np
from scipy.stats import entropy

In [3]:
# Load the abundance data
csv_path = "/home/dermot.kelly/Dermot_analysis/Phd/Paper_2/rumen_microbiome_pipeline/exported/genus_relative_abundance.csv"
genus_rel_abundance = pd.read_csv(csv_path, index_col=0)

# Extract abundances
abundances = genus_rel_abundance.iloc[0].values

# filter out zeros
abundances = abundances[abundances >0]

# Calculate diversity metrics

# Richness
richness = len(abundances)

# Shannon Diversity Index
shannon = entropy(abundances, base=np.e)  # natural log base

# Simpson Diversity Index
simpson = np.sum((abundances / abundances.sum())**2)
simpson_index = 1 - simpson

# Print results
print(f"Richness (number of genera present): {richness}")
print(f"Shannon diversity index: {shannon:.2f}")
print(f"Simpson diversity index: {simpson_index:.2f}")

Richness (number of genera present): 59
Shannon diversity index: 1.63
Simpson diversity index: 0.62
